Need to import libraries only once each time jupiter notebook is launched

In [ ]:
import re
import pandas

from ipyfilechooser import FileChooser
from pathlib import Path
from pandas import DataFrame

Choose raw excel file

In [ ]:
raw_data = FileChooser()
display(raw_data)

In [ ]:
# open excel file as pandas dataframe
form_df = pandas.read_excel(io=raw_data.selected, 
                            sheet_name='Form1')

# remove illegal characters from column names
headers_list = form_df.columns.values.tolist()

def remove_illegal_chars(text):
    text = re.sub('[^A-Za-z0-9_\ ?]+', '', text)
    text.strip()
    return text    
    
headers_list = list(map(remove_illegal_chars, headers_list))
form_df.columns = headers_list

print(f'All columns after cleanup:')
print(*headers_list, sep = "\n")


Type necessary columns that you need for furter parsing. Use single quotes and seperate by comas.

In [ ]:
necessary_columns = [
        'Are you a Big Sky Franchise Team client or were you in the past?',
        'Did you complete the franchise process?',
        'What do you think are Big Skys Franchise Team strengths?',
        'What do you think are Big Skys Franchise Team weaknesses?',
        'How would you rate your overall experience with Big Sky on a scale of 110 1 worst  10 best',
    ]

Filter based on columns

In [ ]:
filtered_form_df = form_df.filter(items=necessary_columns, axis=1)

Save results (choose folder and output filename without '.xlsx' extension)

In [ ]:
target = FileChooser()
display(target)

In [ ]:
target_path = target.selected + '.xlsx'
filtered_form_df.to_excel(excel_writer=target_path, sheet_name='Results')